In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import vapeplot 
%matplotlib inline

#### Technical Challenge Rankings
--------------------------
* tech_med : median technical challenge ranking over each episode
* tech_mean : same as `tech_med` but the mean
* tech : technical challenge ranking for that episode

In [2]:
tech = pd.read_csv("../RESULTS/gbbo.techinical.data.20190907.tsv",sep='\t')
feats = ['tech_mean','tech_med','tech']
tech.head()

,season,baker,index,episode,tech_mean,tech_med,tech,place
0,1,Annetha,6,1,2.00,2.0,2,6
1,1,Annetha,6,2,4.50,4.5,7,6
2,1,Annetha,6,3,3.00,2.0,0,6
3,1,Annetha,6,4,2.25,1.0,0,6
4,1,Annetha,6,5,1.80,0.0,0,6


Data is not scaled or normalized. 

Now apply a MinMax scaler so the minimum value is 0 and the maximum value is 1

In [3]:
from sklearn.preprocessing import MinMaxScaler
mms = tech
scaler = MinMaxScaler()
# fit the scaler
scaler.fit(mms[feats])
# transform values
mms[feats] = scaler.transform(mms[feats])

In [4]:
from sklearn.preprocessing import QuantileTransformer
qua = tech
scaler = QuantileTransformer(
    n_quantiles=10,
    random_state=42,
    ignore_implicit_zeros=True, #sparse matrix
)
# fit the scaler
scaler.fit(qua[feats])
# transform values
qua[feats] = scaler.transform(qua[feats])

/usr/lib/python3.7/site-packages/sklearn/preprocessing/data.py:2142: UserWarning: 'ignore_implicit_zeros' takes effect only with sparse matrix. This parameter has no effect.
  warnings.warn("'ignore_implicit_zeros' takes effect only with"


In [5]:
epi=sorted(list(set(tech['episode'])))
epi

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [ ]:
def clf_